In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import glob
import os
import csv
import numpy as np
import matplotlib

import seaborn as sns
import plotly.plotly as py
sns.set(style="darkgrid", font_scale=1.5)
%matplotlib inline

In [2]:
# Construct the dataframe from BoS Data

amsa2013 = pd.read_excel('oesm13ma/aMSA_M2013_dl.xls')
bos2013 = pd.read_excel('oesm13ma/BOS_M2013_dl.xls')
msa2013_1 = pd.read_excel('oesm13ma/MSA_M2013_dl_1_AK_IN.xls')
msa2013_2 = pd.read_excel('oesm13ma/MSA_M2013_dl_2_KS_NY.xls')
msa2013_3 = pd.read_excel('oesm13ma/MSA_M2013_dl_3_OH_WY.xls')
msa2013 = pd.concat([msa2013_1, msa2013_2, msa2013_3, amsa2013, bos2013], ignore_index=True)
# Clean non-numeric value in 'TOT_EMP'
msa2013.loc[msa2013[[not isinstance(x, int) for x in msa2013['TOT_EMP']]].index, 'TOT_EMP'] = 0

In [3]:
# Construct the dataframe from Data@Work Data
os2013q1 = pd.read_csv('open-skills-data/2013Q1.csv')
os2013q2 = pd.read_csv('open-skills-data/2013Q2.csv')
os2013q3 = pd.read_csv('open-skills-data/2013Q3.csv')
os2013q4 = pd.read_csv('open-skills-data/2013Q4.csv')
os2013 = pd.concat([os2013q1, os2013q2, os2013q3, os2013q4], ignore_index=True)
os2013['major_group'] = pd.Series(list(map(lambda x: x[:2]+'-0000', os2013['soc_code'])))

In [17]:
# Construct Lookup for fips and name
d = os2013[['cbsa_fips','cbsa_name']].drop_duplicates()
lookup = dict(zip(d['cbsa_fips'],d['cbsa_name']))

In [18]:
msa2013.head()

,PRIM_STATE,AREA,AREA_NAME,OCC_CODE,OCC_TITLE,OCC_GROUP,TOT_EMP,EMP_PRSE,JOBS_1000,LOC QUOTIENT,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,AK,11260,"Anchorage, AK",00-0000,All Occupations,total,175540,2.1,1000,1,...,20.76,32.69,45.88,20910,28680,43180,68000,95430,NaN,NaN
1,AK,11260,"Anchorage, AK",11-0000,Management Occupations,major,10370,2.4,59.087,1.2,...,46.09,63.15,85.96,51940,69250,95860,131360,178810,NaN,NaN
2,AK,11260,"Anchorage, AK",11-1011,Chief Executives,detailed,320,4.5,1.817,0.97,...,76.9,#,#,97190,115700,159940,#,#,NaN,NaN
3,AK,11260,"Anchorage, AK",11-1021,General and Operations Managers,detailed,4010,3.2,22.828,1.53,...,45.74,64.77,89.36,52020,66980,95140,134720,185870,NaN,NaN
4,AK,11260,"Anchorage, AK",11-2011,Advertising and Promotions Managers,detailed,40,27.6,0.207,0.96,...,32.49,49.3,56.59,42670,49100,67580,102550,117710,NaN,NaN


In [19]:
os2013.head()

,cbsa_fips,cbsa_name,state_code,soc_code,counts_total,major_group
0,39580,"Raleigh-Cary, NC Metro Area",NC,17-2112.00,8,17-0000
1,12060,"Atlanta-Sandy Springs-Marietta, GA Metro Area",GA,29-1062.00,10,29-0000
2,16700,"Charleston-North Charleston-Summerville, SC Me...",SC,39-9032.00,3,39-0000
3,41700,"San Antonio-New Braunfels, TX Metro Area",TX,29-1051.00,37,29-0000
4,20700,"East Stroudsburg, PA Micro Area",PA,11-2022.00,1,11-0000


# Difference between BoS and Data@Work in 2013

In [23]:
print('How many MSAs?', msa2013['AREA'].unique().shape[0])

How many MSAs? 586


In [54]:
os2013['cbsa_fips'].unique().shape[0]

950

In [24]:
msa2013['PRIM_STATE'].unique().shape[0]

52

In [25]:
os2013['state_code'].unique().shape[0]

52

In [51]:
msa2013[msa2013['OCC_GROUP']=='total']['TOT_EMP'].sum()

169657540

In [39]:
os2013['counts_total'].sum()

3745190

In [60]:
msa2013[msa2013['OCC_GROUP']=='major']['OCC_CODE'].unique().shape

(22,)

In [61]:
os2013['major_group'].unique().shape

(23,)

In [69]:
os2013[os2013['major_group'] == '55-0000'].shape

(754, 6)

In [68]:
os2013['major_group'].unique()

array(['17-0000', '29-0000', '39-0000', '11-0000', '43-0000', '15-0000',
       '41-0000', '53-0000', '37-0000', '27-0000', '25-0000', '51-0000',
       '49-0000', '35-0000', '33-0000', '31-0000', '13-0000', '21-0000',
       '19-0000', '47-0000', '23-0000', '55-0000', '45-0000'], dtype=object)

In [74]:
os2013[(os2013['cbsa_fips']==40380)]['counts_total'].sum()

7441

In [86]:
os2013[(os2013['cbsa_fips']==40380) & (os2013['major_group']=='41-0000')].groupby('soc_code')['counts_total'].sum().to_frame()

,counts_total
soc_code,
41-1011.00,54
41-1012.00,5
41-2011.00,2
41-2021.00,17
41-2022.00,5
41-2031.00,690
41-3011.00,273
41-3021.00,672
41-3031.01,3
